<a href="https://colab.research.google.com/github/Dheeraj1508/EmergencyVehicleClassification/blob/main/Emergency_vehicle_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -q kaggle

In [20]:
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download "abhisheksinghblr/emergency-vehicles-identification"

In [ ]:
!unzip emergency-vehicles-identification.zip

In [24]:
import pandas as pd
import numpy as np

In [25]:
train = pd.read_csv('Emergency_Vehicles/train.csv')

In [27]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.mobilenet_v3 import preprocess_input,MobileNetV3Large
# from keras.applications.vgg16 import 
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
vgg = MobileNetV3Large(input_shape=[224,224] + [3], weights='imagenet', include_top=False)
for layer in vgg.layers:
  layer.trainable = False

In [61]:
x = Flatten()(vgg.output)
x = Dense(128,activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)

In [68]:
from keras import optimizers
from tensorflow import keras

sgd = keras.optimizers.Adam(learning_rate=0.0001,decay=0.000001)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [69]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


In [70]:
train.emergency_or_not=train.emergency_or_not.astype(str)

In [71]:
train_set = train_datagen.flow_from_dataframe(train,directory='Emergency_Vehicles/train',x_col='image_names',y_col='emergency_or_not',target_size=(224,224),class_mode='binary',batch_size=32)


Found 1646 validated image filenames belonging to 2 classes.


In [82]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
my_callbacks = [
               ModelCheckpoint('my_model.h5', monitor = 'accuracy',verbose = 1,save_weights_only=True, save_best_only = True,mode="max"),
               EarlyStopping(monitor='loss', patience=8, verbose=0, mode='min'),
               ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=3, verbose=1, mode='min',min_delta=1e-4)
]

In [85]:
model.fit( train_set,
  epochs=5,callbacks=[my_callbacks])

Epoch 1/5
52/52 [==============================] - 18s 346ms/step - loss: 0.0441 - accuracy: 0.9848

Epoch 00001: accuracy did not improve from 0.98967
Epoch 2/5
52/52 [==============================] - 18s 345ms/step - loss: 0.0438 - accuracy: 0.9836

Epoch 00002: accuracy did not improve from 0.98967
Epoch 3/5
52/52 [==============================] - 18s 345ms/step - loss: 0.0437 - accuracy: 0.9872

Epoch 00003: accuracy did not improve from 0.98967
Epoch 4/5
52/52 [==============================] - 18s 343ms/step - loss: 0.0457 - accuracy: 0.9842

Epoch 00004: accuracy did not improve from 0.98967
Epoch 5/5
52/52 [==============================] - 18s 344ms/step - loss: 0.0506 - accuracy: 0.9830

Epoch 00005: accuracy did not improve from 0.98967
